In [1]:
import mlflow
import torch
import os



# import path
workingdir = os.getcwd()
os.chdir(workingdir)
os.chdir("../")
import src.pipeline.rankme_evaluator as RankMe
import src.pipeline.finetune as finetune
from src.models.encoder_projector import EncoderProjector
from src.utils.load_dataset import load_dataset




In [2]:
all_runs = mlflow.search_runs()
# filter out runs where status is failed
all_successful_runs = all_runs[all_runs['status'] == 'FINISHED']
run_id = all_runs.sample(1)['run_id'].values[0]

In [19]:
#run_id = "b47321ebb5dc4b919198d9be17a1f782"


model_uri = f"runs:/{run_id}/pretraining_model/"

os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:8080"
loaded_model = mlflow.pytorch.load_model(model_uri)

loaded_model.eval()

EncoderProjector(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [4]:
cifar100 = load_dataset("_datasets", "CIFAR100")

Files already downloaded and verified


In [5]:
relpath = f"mlartifacts/0/{run_id}/artifacts/pretraining_model/data/model.pth"

rank = RankMe.rank_me(loaded_model, cifar100, device="cpu")

print("estimated rank", rank)

Doing forward pass on full dataset


100%|██████████| 781/781 [04:01<00:00,  3.23it/s]


time to compute rank 4.028825759887695
estimated rank tensor(264.8275)


In [9]:
epochs = 1

In [7]:

dataset_path = "/Users/ntforsberg/dev/courses/adl/rankme-reprod/_datasets"

dataset_name = "CIFAR100"

epochs = 1

finetune.finetune_pipeline(loaded_model, dataset_path, dataset_name, epochs=epochs)

Files already downloaded and verified
Files already downloaded and verified
[1,   100] loss: 4.810
[1,   200] loss: 4.266
[1,   300] loss: 4.122
[1,   400] loss: 4.095
[1,   500] loss: 4.040
[1,   600] loss: 4.023
[1,   700] loss: 3.976
Finished Training
Accuracy of the network on the 10000 test images: 15 %


15.242

In [10]:
accuracy = 15.242



with mlflow.start_run(run_id=run_id):
    mlflow.log_metric("rank", rank)
    mlflow.log_metric(f"{dataset_name}_accuracy", accuracy)
    mlflow.log_param(f"{dataset_name}_epochs", epochs)

    mlflow.end_run()


